In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set(style="darkgrid")

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
def rmse(y_true,y_pred):
    return mean_squared_error(y_true=y_true,y_pred=y_pred) ** 0.5

In [2]:
train = pd.read_csv('./data/train/保定2016年.csv',encoding = 'gb2312')
test  = pd.read_csv('./data/test/石家庄20160701-20170701极值改为缺失值.csv',encoding = 'gb2312')

In [3]:
# 分析数据 发现该数据更适合先用回归模型
data = pd.concat([train,test]).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      732 non-null    int64  
 1   日期      732 non-null    object 
 2   AQI     732 non-null    int64  
 3   质量等级    732 non-null    object 
 4   PM2.5   726 non-null    float64
 5   PM10    726 non-null    float64
 6   SO2     732 non-null    int64  
 7   CO      732 non-null    float64
 8   NO2     732 non-null    int64  
 9   O3_8h   732 non-null    int64  
 10  IPRC    366 non-null    float64
dtypes: float64(4), int64(5), object(2)
memory usage: 63.0+ KB


In [4]:
# # label变换
# w1 = 150
# train['new_label'] = train['IPRC'] - train['AQI'] / w1

In [5]:
# 此处因为我上传的原始数据有问题，之前想用其它值来代替测量值为0
data['PM2.5'] = data['PM2.5'].replace({np.NaN:0})
data['PM10'] = data['PM10'].replace({np.NaN:0})

In [6]:
data.describe()

,id,AQI,PM2.5,PM10,SO2,CO,NO2,O3_8h,IPRC
count,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,366.000000
mean,366.500000,141.923497,100.952186,164.239071,39.486339,1.667896,57.838798,93.172131,0.891796
std,211.454487,90.614468,88.592253,118.379362,28.373435,1.399841,27.653489,58.582951,0.557107
min,1.000000,36.000000,0.000000,0.000000,5.000000,0.200000,13.000000,5.000000,0.214400
25%,183.750000,78.000000,41.000000,80.750000,19.000000,0.800000,37.000000,44.750000,0.486215
50%,366.500000,114.000000,72.000000,133.000000,32.000000,1.100000,51.000000,88.000000,0.749276
75%,549.250000,182.000000,133.000000,204.500000,50.000000,2.000000,71.250000,132.250000,1.170173
max,732.000000,500.000000,621.000000,866.000000,153.000000,10.400000,183.000000,278.000000,3.316942


In [7]:
# # 对PM2.5区段进行划分 中国标准
# def IAQI_PM25(x):  
#     if x >=0 and x<=35:
#         y = 50/35 * x
#         return y
#     elif x>35 and x<=75:
#         y = 49/39 * (x-36) + 51
#         return y 
#     elif x>75 and x<=115:
#         y = 49/39 * (x-76) + 101
#         return y
#     elif x>115 and x<=150:
#         y = 49/34 * (x-116) + 151
#         return y
#     elif x>150 and x<=250:
#         y = 99/99 * (x-151) + 201
#         return y 
#     elif x>250 and x<=350:
#         y = 99/99 * (x-251) + 301
#         return y
#     elif x>350 and x<=500:
#         y = 99/149 * (x-351) + 401
#         return y 
#     else:  
#         return 500 
    
# data['IAQI_PM25'] = data['PM2.5'].apply(lambda x: round(IAQI_PM25(x)))
# #核密度分布图
# sns.distplot(data["IAQI_PM25"])
# print(data["IAQI_PM25"].skew())

In [8]:
# 对PM2.5区段进行划分 美国标准 对小数进行修改
def IAQI_PM25_US(x):  
    if x >=0 and x<=15:
        y = 50/15 * x
        return y
    elif x>15 and x<=40:
        y = 49/24 * (x-16) + 51
        return y 
    elif x>40 and x<=65:
        y = 49/24 * (x-41) + 101
        return y
    elif x>65 and x<=150:
        y = 49/84 * (x-66) + 151
        return y
    elif x>150 and x<=250:
        y = 99/99 * (x-151) + 201
        return y 
    elif x>250 and x<=350:
        y = 99/99 * (x-251) + 301
        return y
    elif x>350 and x<=500:
        y = 99/149 * (x-351) + 401
        return y 
    else:  
        return 500 
    
data['IAQI_PM25'] = data['PM2.5'].apply(lambda x: round(IAQI_PM25_US(x)))
#核密度分布图
sns.distplot(data["IAQI_PM25"])
print(data["IAQI_PM25"].skew())

1.3244020129550784


In [9]:
# 对PM10区段进行划分 中国标准
def IAQI_PM10(x):  
    if x >=0 and x<=50:
        y = 50/50 * x
        return y
    elif x>50 and x<=150:
        y = 49/99 * (x-51) + 51
        return y 
    elif x>150 and x<=250:
        y = 49/99 * (x-151) + 101
        return y
    elif x>250 and x<=350:
        y = 49/99 * (x-251) + 151
        return y
    elif x>350 and x<=420:
        y = 99/69 * (x-351) + 201
        return y 
    elif x>420 and x<=500:
        y = 99/79 * (x-421) + 301
        return y
    elif x>500 and x<=600:
        y = 99/99 * (x-501) + 401
        return y 
    else:  
        return 500
    
data['IAQI_PM10'] = data['PM10'].apply(lambda x: round(IAQI_PM10(x)))
#核密度分布图
sns.distplot(data["IAQI_PM10"])
print(data["IAQI_PM10"].skew())

2.683676756590901


In [10]:
# # 对PM10区段进行划分 美国标准
# def IAQI_PM10_US(x):  
#     if x >=0 and x<=54:
#         y = 50/54 * x
#         return y
#     elif x>54 and x<=154:
#         y = 50/100 * (x-54) + 50
#         return y 
#     elif x>154 and x<=254:
#         y = 50/100 * (x-154) + 100
#         return y
#     elif x>254 and x<=354:
#         y = 50/100 * (x-254) + 150
#         return y
#     elif x>354 and x<=424:
#         y = 100/70 * (x-354) + 200
#         return y 
#     elif x>424 and x<=504:
#         y = 100/80 * (x-424) + 300
#         return y
#     elif x>504 and x<=604:
#         y = 100/100 * (x-504) + 400
#         return y 
#     else:  
#         return 500
    
# data['IAQI_PM10'] = data['PM10'].apply(lambda x: IAQI_PM10_US(x))
# #核密度分布图
# sns.distplot(data["IAQI_PM10"])
# print(data["IAQI_PM10"].skew())

In [11]:
# # 中国标准
# def IAQI_SO2(x):  
#     if x >=0 and x<=50:
#         y = 50/50 * x
#         return y
#     elif x>50 and x<=150:
#         y = 49/99 * (x-51) + 51
#         return y 
#     elif x>150 and x<=475:
#         y = 49/324 * (x-151) + 101
#         return y
#     elif x>475 and x<=800:
#         y = 49/324 * (x-476) + 151
#         return y
#     elif x>800 and x<=1600:
#         y = 99/799 * (x-801) + 201
#         return y 
#     elif x>1600 and x<=2100:
#         y = 99/499 * (x-1601) + 301
#         return y
#     elif x>2100 and x<=2620:
#         y = 99/519 * (x-2101) + 401
#         return y 
#     else:  
#         return 500
    
# data['IAQI_SO2'] = data['SO2'].apply(lambda x: round(IAQI_SO2(x)))
# #核密度分布图
# sns.distplot(data["IAQI_SO2"])
# print(data["IAQI_SO2"].skew())

In [12]:
# 美国标准
def IAQI_SO2_US(x):  
    if x >=0 and x<=92:
        y = 50/92 * x
        return y
    elif x>92 and x<=200:
        y = 49/107 * (x-93) + 51
        return y 
    elif x>200 and x<=480:
        y = 49/279 * (x-201) + 101
        return y
    elif x>480 and x<=800:
        y = 49/319 * (x-481) + 151
        return y
    elif x>800 and x<=1580:
        y = 99/779 * (x-801) + 201
        return y 
    elif x>1580 and x<=2100:
        y = 99/519 * (x-1581) + 301
        return y
    
data['IAQI_SO2'] = data['SO2'].apply(lambda x: round(IAQI_SO2_US(x)))
#核密度分布图
sns.distplot(data["IAQI_SO2"])
print(data["IAQI_SO2"].skew())

1.2566357021886638


In [13]:
def IAQI_NO2(x):  
    if x >=0 and x<=40:
        y = 50/40 * x
        return y
    elif x>40 and x<=80:
        y = 49/39 * (x-41) + 51
        return y 
    elif x>80 and x<=180:
        y = 49/99 * (x-81) + 101
        return y
    elif x>180 and x<=280:
        y = 49/99 * (x-181) + 151
        return y
    elif x>280 and x<=565:
        y = 99/284 * (x-281) + 201
        return y 
    elif x>565 and x<=750:
        y = 99/184 * (x-566) + 301
        return y
    elif x>750 and x<=940:
        y = 99/189 * (x-751) + 401
        return y 
    else:  
        return 500
    
data['IAQI_NO2'] = data['NO2'].apply(lambda x: round(IAQI_NO2(x)))
#核密度分布图
sns.distplot(data["IAQI_NO2"])
print(data["IAQI_NO2"].skew())

0.5365729135229688


In [14]:
# # 美国无此项指标，我国根据自身需要增加了 NO2-24 h
# def IAQI_NO2_US(x):  
#     if x >=0 and x<=40:
#         y = 50/40 * x
#         return y
#     elif x>40 and x<=80:
#         y = 50/40 * (x-40) + 50
#         return y 
#     elif x>80 and x<=180:
#         y = 50/100 * (x-80) + 100
#         return y
#     elif x>180 and x<=280:
#         y = 50/100 * (x-180) + 150
#         return y
#     elif x>280 and x<=565:
#         y = 100/285 * (x-280) + 200
#         return y 
#     elif x>565 and x<=750:
#         y = 100/185 * (x-565) + 300
#         return y
#     elif x>750 and x<=940:
#         y = 100/190 * (x-750) + 400
#         return y 
#     else:  
#         return 500
    
# data['IAQI_NO2_US'] = data['NO2'].apply(lambda x: IAQI_NO2_US(x))
# #核密度分布图
# sns.distplot(data["IAQI_NO2_US"])
# print(data["IAQI_NO2_US"].skew())

In [15]:
def IAQI_CO(x):  
    if x >=0 and x<=2:
        y = 50/2 * x
        return y
    elif x>2 and x<=4:
        y = 49/1.9 * (x-2.1) + 51
        return y  
    else:
        y = 49/9.9 * (x-4.1) + 101
        return y
    
data['IAQI_CO'] = data['CO'].apply(lambda x: round(IAQI_CO(x)))
#核密度分布图
sns.distplot(data["IAQI_CO"])
print(data["IAQI_CO"].skew())

1.3733868586546452


In [16]:
# # 美国此项标准不如我国严格，沿用我国标准
# def IAQI_CO_US(x):  
#     if x >=0 and x<=2:
#         y = 50/2 * x
#         return y
#     elif x>2 and x<=4:
#         y = 50/2 * (x-2) + 50
#         return y  
#     else:
#         y = 50/10 * (x-4) + 100
#         return y
    
# data['IAQI_CO_US'] = data['CO'].apply(lambda x: IAQI_CO_US(x))
# #核密度分布图
# sns.distplot(data["IAQI_CO_US"])
# print(data["IAQI_CO_US"].skew())

In [17]:
# # 中国标准
# def IAQI_O3(x):  
#     if x >=0 and x<=100:
#         y = 50/100 * x
#         return y
#     elif x>100 and x<=160:
#         y = 49/59 * (x-101) + 51
#         return y 
#     elif x>160 and x<=215:
#         y = 49/54 * (x-161) + 101
#         return y
#     elif x>215 and x<=265:
#         y = 49/49 * (x-216) + 151
#         return y
#     else: 
#         y = 99/534 * (x-266) + 201
#         return y
    
    
# data['IAQI_O3'] = data['O3_8h'].apply(lambda x: round(IAQI_O3(x)))
# #核密度分布图
# sns.distplot(data["IAQI_O3"])
# print(data["IAQI_O3"].skew())

In [18]:
# 中国标准 参考美国标准修改
def IAQI_O3(x):  
    if x >=0 and x<=100:
        y = 50/100 * x
        return y
    elif x>100 and x<=150:
        y = 49/49 * (x-101) + 51
        return y 
    elif x>150 and x<=190:
        y = 49/39 * (x-151) + 101
        return y
    elif x>190 and x<=230:
        y = 49/39 * (x-191) + 151
        return y
    else: 
        y = 99/499 * (x-231) + 201
        return y
    
    
data['IAQI_O3'] = data['O3_8h'].apply(lambda x: round(IAQI_O3(x)))
#核密度分布图
sns.distplot(data["IAQI_O3"])
print(data["IAQI_O3"].skew())

1.210223037368436


In [19]:
# # 美国标准
# def IAQI_O3_US(x):  
#     if x >=0 and x<=120:
#         y = 50/120 * x
#         return y
#     elif x>120 and x<=150:
#         y = 49/29 * (x-121) + 51
#         return y 
#     elif x>150 and x<=190:
#         y = 49/39 * (x-151) + 101
#         return y
#     elif x>190 and x<=230:
#         y = 49/39 * (x-191) + 151
#         return y
#     else: 
#         y = 99/499 * (x-231) + 201
#         return y
    
    
# data['IAQI_O3'] = data['O3_8h'].apply(lambda x: round(IAQI_O3_US(x)))
# #核密度分布图
# sns.distplot(data["IAQI_O3"])
# print(data["IAQI_O3"].skew())

In [20]:
data_onehot = pd.get_dummies(data['质量等级'])

data = pd.concat([data,data_onehot],axis=1)
data.head()

,id,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3_8h,...,IAQI_SO2,IAQI_NO2,IAQI_CO,IAQI_O3,严重污染,中度污染,优,良,轻度污染,重度污染
0,1,2016/1/1,293,重度污染,243.0,324.0,122,6.1,149,12,...,64,135,111,6,0,0,0,0,0,1
1,2,2016/1/2,430,严重污染,395.0,517.0,138,7.5,180,18,...,72,150,118,9,1,0,0,0,0,0
2,3,2016/1/3,332,严重污染,282.0,405.0,72,6.3,130,10,...,39,125,112,5,1,0,0,0,0,0
3,4,2016/1/4,204,重度污染,154.0,237.0,73,3.5,72,34,...,40,90,87,17,0,0,0,0,0,1
4,5,2016/1/5,169,中度污染,128.0,186.0,99,3.2,66,39,...,54,82,79,20,0,1,0,0,0,0


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         732 non-null    int64  
 1   日期         732 non-null    object 
 2   AQI        732 non-null    int64  
 3   质量等级       732 non-null    object 
 4   PM2.5      732 non-null    float64
 5   PM10       732 non-null    float64
 6   SO2        732 non-null    int64  
 7   CO         732 non-null    float64
 8   NO2        732 non-null    int64  
 9   O3_8h      732 non-null    int64  
 10  IPRC       366 non-null    float64
 11  IAQI_PM25  732 non-null    int64  
 12  IAQI_PM10  732 non-null    int64  
 13  IAQI_SO2   732 non-null    int64  
 14  IAQI_NO2   732 non-null    int64  
 15  IAQI_CO    732 non-null    int64  
 16  IAQI_O3    732 non-null    int64  
 17  严重污染       732 non-null    uint8  
 18  中度污染       732 non-null    uint8  
 19  优          732 non-null    uint8  
 20  良         

#### 首要污染物 
存在两种计算主要污染物的方式

1.依据IAQI

2.依据ci/si

In [22]:
temp = data.iloc[:,11:17]
temp.head()

,IAQI_PM25,IAQI_PM10,IAQI_SO2,IAQI_NO2,IAQI_CO,IAQI_O3
0,293,187,64,135,111,6
1,430,417,72,150,118,9
2,332,278,39,125,112,5
3,204,144,40,90,87,17
4,187,118,54,82,79,20


In [23]:
temp = data.iloc[:,11:17]
data['max_idx'] = temp.idxmax(axis=1) #求一行的最大值对应的索引
data['max_val']= temp.max(axis=1) #取出该最大值
data.head()

,id,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3_8h,...,IAQI_CO,IAQI_O3,严重污染,中度污染,优,良,轻度污染,重度污染,max_idx,max_val
0,1,2016/1/1,293,重度污染,243.0,324.0,122,6.1,149,12,...,111,6,0,0,0,0,0,1,IAQI_PM25,293
1,2,2016/1/2,430,严重污染,395.0,517.0,138,7.5,180,18,...,118,9,1,0,0,0,0,0,IAQI_PM25,430
2,3,2016/1/3,332,严重污染,282.0,405.0,72,6.3,130,10,...,112,5,1,0,0,0,0,0,IAQI_PM25,332
3,4,2016/1/4,204,重度污染,154.0,237.0,73,3.5,72,34,...,87,17,0,0,0,0,0,1,IAQI_PM25,204
4,5,2016/1/5,169,中度污染,128.0,186.0,99,3.2,66,39,...,79,20,0,1,0,0,0,0,IAQI_PM25,187


In [24]:
data['max_idx'].unique()

array(['IAQI_PM25', 'IAQI_O3', 'IAQI_PM10', 'IAQI_NO2'], dtype=object)

In [25]:
# data.loc[data.AQI<=50,'max_idx'] = 'No_polution'

In [26]:
data['max_idx'] = data['max_idx'].replace('IAQI_PM25', 'PM2.5_P') 
data['max_idx'] = data['max_idx'].replace('IAQI_PM10', 'PM10_P') 
data['max_idx'] = data['max_idx'].replace('IAQI_O3', 'O3_P') 
data['max_idx'] = data['max_idx'].replace('IAQI_NO2', 'NO2_P')

In [27]:
data_onehot = pd.get_dummies(data['max_idx'])

data = pd.concat([data,data_onehot],axis=1)
data.head()

,id,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3_8h,...,优,良,轻度污染,重度污染,max_idx,max_val,NO2_P,O3_P,PM10_P,PM2.5_P
0,1,2016/1/1,293,重度污染,243.0,324.0,122,6.1,149,12,...,0,0,0,1,PM2.5_P,293,0,0,0,1
1,2,2016/1/2,430,严重污染,395.0,517.0,138,7.5,180,18,...,0,0,0,0,PM2.5_P,430,0,0,0,1
2,3,2016/1/3,332,严重污染,282.0,405.0,72,6.3,130,10,...,0,0,0,0,PM2.5_P,332,0,0,0,1
3,4,2016/1/4,204,重度污染,154.0,237.0,73,3.5,72,34,...,0,0,0,1,PM2.5_P,204,0,0,0,1
4,5,2016/1/5,169,中度污染,128.0,186.0,99,3.2,66,39,...,0,0,0,0,PM2.5_P,187,0,0,0,1


#### 是否高于年平均二级指标

In [28]:
data['PM2.5_year'] =data['PM2.5'].apply(lambda x: 0 if x <=35 else 1)
data['PM10_year'] =data['PM10'].apply(lambda x: 0 if x <=70 else 1)
data['SO2_year'] =data['SO2'].apply(lambda x: 0 if x <=60 else 1)
data['NO2_year'] =data['NO2'].apply(lambda x: 0 if x <=40 else 1)
# data['CO_year'] =data['CO'].apply(lambda x: 0 if x <=4 else 1)
# data['O3_8h_year'] =data['O3_8h'].apply(lambda x: 0 if x <=160 else 1)

#### 是否高于24小时平均二级指标 O3 8小时指标

In [29]:
data['PM2.5_day'] =data['PM2.5'].apply(lambda x: 0 if x <=75 else 1)
data['PM10_day'] =data['PM10'].apply(lambda x: 0 if x <=150 else 1)
data['SO2_day'] =data['SO2'].apply(lambda x: 0 if x <=150 else 1)
data['NO2_day'] =data['NO2'].apply(lambda x: 0 if x <=80 else 1)
data['CO_day'] =data['CO'].apply(lambda x: 0 if x <=4 else 1)
data['O3_8h_day'] =data['O3_8h'].apply(lambda x: 0 if x <=160 else 1)

#### 超标污染物

In [30]:
# IAQI_feature = ['IAQI_PM25','IAQI_PM10','IAQI_SO2','IAQI_NO2','IAQI_CO','IAQI_O3']

# for f in IAQI_feature:
#     data['over_{}'.format(f)] = data[f].apply(lambda x: 0 if x <=100 else 1)

#### 单项质量指数

In [31]:
data['SO2_index'] = data['SO2'] / 60

data['NO2_index'] = data['NO2'] / 40

data['PM10_index'] = data['PM10'] / 70

data['PM2.5_index'] = data['PM2.5'] / 35

data['CO_index'] = data['CO'] / 4

data['O3_8h_index'] = data['O3_8h'] / 160

data['sum_index'] = data['SO2_index'] + data['NO2_index'] + data['PM10_index'] + data['PM2.5_index'] + data['CO_index'] + data['O3_8h_index']


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 46 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           732 non-null    int64  
 1   日期           732 non-null    object 
 2   AQI          732 non-null    int64  
 3   质量等级         732 non-null    object 
 4   PM2.5        732 non-null    float64
 5   PM10         732 non-null    float64
 6   SO2          732 non-null    int64  
 7   CO           732 non-null    float64
 8   NO2          732 non-null    int64  
 9   O3_8h        732 non-null    int64  
 10  IPRC         366 non-null    float64
 11  IAQI_PM25    732 non-null    int64  
 12  IAQI_PM10    732 non-null    int64  
 13  IAQI_SO2     732 non-null    int64  
 14  IAQI_NO2     732 non-null    int64  
 15  IAQI_CO      732 non-null    int64  
 16  IAQI_O3      732 non-null    int64  
 17  严重污染         732 non-null    uint8  
 18  中度污染         732 non-null    uint8  
 19  优       

In [33]:
# 保留两位小数
index_feature = ['PM2.5_index', 'PM10_index', 'SO2_index', 'NO2_index', 'CO_index', 'O3_8h_index','sum_index']
for f in index_feature:
    data[f] = data[f].apply(lambda x: round(x,2))

#### 第二种计算首要污染物的方式

In [34]:
temp = data.iloc[:,33:39]
data['I_max_idx'] = temp.idxmax(axis=1) #求一行的最大值对应的索引
data['I_max_val']= temp.max(axis=1) #取出该最大值
data.head()

,id,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3_8h,...,O3_8h_day,SO2_index,NO2_index,PM10_index,PM2.5_index,CO_index,O3_8h_index,sum_index,I_max_idx,I_max_val
0,1,2016/1/1,293,重度污染,243.0,324.0,122,6.1,149,12,...,0,2.03,3.73,4.63,6.94,1.52,0.07,18.93,PM2.5_day,1
1,2,2016/1/2,430,严重污染,395.0,517.0,138,7.5,180,18,...,0,2.30,4.50,7.39,11.29,1.88,0.11,27.46,PM2.5_day,1
2,3,2016/1/3,332,严重污染,282.0,405.0,72,6.3,130,10,...,0,1.20,3.25,5.79,8.06,1.57,0.06,19.93,PM2.5_day,1
3,4,2016/1/4,204,重度污染,154.0,237.0,73,3.5,72,34,...,0,1.22,1.80,3.39,4.40,0.88,0.21,11.89,PM2.5_day,1
4,5,2016/1/5,169,中度污染,128.0,186.0,99,3.2,66,39,...,0,1.65,1.65,2.66,3.66,0.80,0.24,10.66,PM2.5_day,1


In [35]:
data['I_max_idx'].unique()

array(['PM2.5_day', 'PM10_day', 'O3_8h_day'], dtype=object)

In [36]:
data['I_max_idx'] = data['I_max_idx'].replace('PM2.5_index', 'PM2.5_P') 
data['I_max_idx'] = data['I_max_idx'].replace('PM10_index', 'PM10_P') 
data['I_max_idx'] = data['I_max_idx'].replace('O3_8h_index', 'O3_P') 
data['I_max_idx'] = data['I_max_idx'].replace('NO2_index', 'NO2_P')

In [37]:
data_onehot = pd.get_dummies(data['I_max_idx'])

data = pd.concat([data,data_onehot],axis=1)
data.head()

,id,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3_8h,...,PM10_index,PM2.5_index,CO_index,O3_8h_index,sum_index,I_max_idx,I_max_val,O3_8h_day,PM10_day,PM2.5_day
0,1,2016/1/1,293,重度污染,243.0,324.0,122,6.1,149,12,...,4.63,6.94,1.52,0.07,18.93,PM2.5_day,1,0,0,1
1,2,2016/1/2,430,严重污染,395.0,517.0,138,7.5,180,18,...,7.39,11.29,1.88,0.11,27.46,PM2.5_day,1,0,0,1
2,3,2016/1/3,332,严重污染,282.0,405.0,72,6.3,130,10,...,5.79,8.06,1.57,0.06,19.93,PM2.5_day,1,0,0,1
3,4,2016/1/4,204,重度污染,154.0,237.0,73,3.5,72,34,...,3.39,4.40,0.88,0.21,11.89,PM2.5_day,1,0,0,1
4,5,2016/1/5,169,中度污染,128.0,186.0,99,3.2,66,39,...,2.66,3.66,0.80,0.24,10.66,PM2.5_day,1,0,0,1


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 51 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           732 non-null    int64  
 1   日期           732 non-null    object 
 2   AQI          732 non-null    int64  
 3   质量等级         732 non-null    object 
 4   PM2.5        732 non-null    float64
 5   PM10         732 non-null    float64
 6   SO2          732 non-null    int64  
 7   CO           732 non-null    float64
 8   NO2          732 non-null    int64  
 9   O3_8h        732 non-null    int64  
 10  IPRC         366 non-null    float64
 11  IAQI_PM25    732 non-null    int64  
 12  IAQI_PM10    732 non-null    int64  
 13  IAQI_SO2     732 non-null    int64  
 14  IAQI_NO2     732 non-null    int64  
 15  IAQI_CO      732 non-null    int64  
 16  IAQI_O3      732 non-null    int64  
 17  严重污染         732 non-null    uint8  
 18  中度污染         732 non-null    uint8  
 19  优       

In [39]:
feature = [
    'AQI', 
    'IAQI_PM25', 'IAQI_PM10', 'IAQI_SO2', 'IAQI_CO', 'IAQI_NO2', 'IAQI_O3',
#     'IAQI_PM25_US', 'IAQI_PM10_US', 'IAQI_SO2_US', 'IAQI_CO_US', 'IAQI_NO2_US', 'IAQI_O3_US',
    'PM2.5_index', 'PM10_index', 'SO2_index', 'NO2_index', 'CO_index', 'O3_8h_index','sum_index',
    'PM2.5_year', 'PM10_year', 'SO2_year',  'NO2_year',
    'PM2.5_day', 'PM10_day', 'SO2_day', 'NO2_day', 'CO_day', 'O3_8h_day',
    'PM2.5_P','PM10_P','O3_P','NO2_P',
#     'max_val',
    '优', '良', '轻度污染',  '中度污染', '重度污染', '严重污染',
]

label = 'IPRC'

In [40]:
train = data[:train.shape[0]]
test  = data[train.shape[0]:]

oof_train = np.zeros((train.shape[0],))
oof_test  = np.zeros((test.shape[0],))

In [41]:
# from sklearn.ensemble import RandomForestRegressor
# from lightgbm.sklearn import LGBMRegressor
# from sklearn.linear_model import Ridge, Lasso

kf = KFold(n_splits=5,random_state=42,shuffle=True)
for index,(tr_index,vl_index) in enumerate(kf.split(train)):
    X_train,X_valid = train.iloc[tr_index][feature].values,train.iloc[vl_index][feature].values
    y_train,y_valid = train.iloc[tr_index][label],train.iloc[vl_index][label]

    model = LinearRegression()
#     model = Ridge(alpha=0.05)
#     model = Lasso(alpha=0.001)
#     model = RandomForestRegressor(n_estimators=10, criterion = 'mse', random_state=42)
#     model = LGBMRegressor()
    model.fit(X_train,y_train)

    oof_train[vl_index] = model.predict(X_valid)
    oof_test = oof_test + model.predict(test[feature].values) / kf.n_splits

In [166]:
r = rmse(train[label],oof_train)

submit = test[['日期']]
submit['IPRC'] = oof_test

submit.columns = ['date','IPRC']
submit[['date','IPRC']].to_csv('./result/LR_{}.csv'.format(str(r).replace('.','_')),index=False)